In [45]:
%pip install nltk sentence_transformers

ERROR: torchvision 0.15.1 has requirement torch==2.0.0, but you'll have torch 1.13.1 which is incompatible.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py install for sentence-transformers: started
    Running setup.py install for sentence-transformers: finished with status 'done'
  Using cached https://files.pythonhosted.org/packages/20/9c/f07bd70d128fdb107bc02a0c702b9058b4fe147d0ba67b5a0f4c3cf15a54/sentence-transformers-2.2.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/5b/fb/478a0460ae2843dd2fc7a7f9ddcd8bb033ae21eb968df6a8cbe8094a28bc/scikit_learn-1.2.2-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/72/31/92d0ada03abf4d2b98fdaa64f10d760225ac2a9a398c0fd7ba6f3660b17b/sentencepiece-0.1.97-cp38-cp38-win_amd64.whl
  Running setup.py install for sentence-transformers: started
    Running setup.py install for sentence-transformers: finished with status 'done'
  Using cached https://files.pythonhosted.org/packages/20/9c/f07bd70d128fdb107bc02a0c702b9058b4fe147d0ba67b5a0f4c3cf15a54/sentence-transformers-2.2.2.tar.gz
  Running setup.py install for sentence-transformers: started


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [46]:
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
from FunctionExtract import core_extractor
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import os
from sentence_transformers import SentenceTransformer, util

In [50]:
class Comparison():
    def __init__(this, ref, gen):
        this.ref = ref
        this.gen = gen

    def compareBLEU(this):
        refsplit = this.ref.split(" ")
        gensplit = this.gen.split(" ")
        
        score = sentence_bleu([refsplit],gensplit)
        this.similarity = score
        return(score)
    
    def compareHF(this):
        model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

        ref_embedding = model.encode(this.ref, convert_to_tensor = True)
        gen_embedding = model.encode(this.gen, convert_to_tensor = True)

        return(util.pytorch_cos_sim(ref_embedding,gen_embedding)[0][0])

class comparisonCorpus():
    def __init__(this):
        this.comparisonpairs = []

    def addComparison(this,comparison):
        this.comparisonpairs.append(comparison)
    
    def generateMetrics(this):
        BLEUscores = []
        HFscores = []
        lengthAnomoly = []
        
        for comparison in this.comparisonpairs:
            BLEUscores.append(comparison.compareBLEU())
            HFscores.append(comparison.compareHF())
            lengthAnomoly.append(np.abs(len(comparison.ref)-len(comparison.gen)))
        
        print("BLEU similarity metrics for dataset: ")
        print("Mean: ",np.mean(BLEUscores))
        print("Median: ",np.median(BLEUscores))
        print("HF Sentence-Transformer similarity metrics for dataset: ")
        print("Mean: ",np.mean(HFscores))
        print("Median: ",np.median(HFscores))
        print("Length anomoly metrics for dataset: ")
        print("Mean: ",np.mean(lengthAnomoly))
        print("Median: ",np.median(lengthAnomoly))
        print("")

In [57]:
import csv
import sys
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

CODE_START = "CODE:\n"
CODE_END = "CODE_END"
COMMENT_START = "COMMENT:\n"
COMMENT_END = "COMMENT_END"

pairs = []

with open("train.csv","r", encoding="utf-8") as f:
    reader = csv.reader(f)
    rownumber = 0
    for row in reader:
        if(rownumber > 0):
            cell_contents = ",".join(row)
            #print("cell contents: ", cell_contents)
            code_start = cell_contents.index(CODE_START)
            code_end = cell_contents.index(CODE_END)
            comment_start = cell_contents.index(COMMENT_START)
            comment_end = cell_contents.index(COMMENT_END)
            code = cell_contents[code_start + len(CODE_START):code_end]
            comment = cell_contents[comment_start + len(COMMENT_START):comment_end]
            pairs.append((code,comment))
        rownumber += 1


In [ ]:
cc1 = comparisonCorpus()

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

#model = T5ForConditionalGeneration.from_pretrained(".")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base-multi-sum")

for i in range(100):
    print(i)
    code = pairs[i][0]
    comment = pairs[i][1]

    input_ids = tokenizer(code,return_tensors="pt").input_ids
    generated_ids = model.generate(input_ids, max_length=512)
    gencomment = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    cc1.addComparison(Comparison(comment,gencomment))

In [ ]:
cc2 = comparisonCorpus()

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')

#model = T5ForConditionalGeneration.from_pretrained(".")
model = T5ForConditionalGeneration.from_pretrained(".")

for i in range(100):
    print(i)
    code = pairs[i][0]
    comment = pairs[i][1]

    input_ids = tokenizer(code,return_tensors="pt").input_ids
    generated_ids = model.generate(input_ids, max_length=512)
    gencomment = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    cc2.addComparison(Comparison(comment,gencomment))

In [54]:
cc.generateMetrics()

c:\Users\Anthony Pierson\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Anthony Pierson\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Anthony Pierson\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how m

BLEU similarity metrics for dataset: 
Mean:  0.02856299175538744
Median:  9.674272450044849e-239
HF Sentence-Transformer similarity metrics for dataset: 
Mean:  0.44830078
Median:  0.40915382
Length anomoly metrics for dataset: 
Mean:  74.9
Median:  52.0

